In [1]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import pynance as pn

In [ ]:
# ---
# title: Quantitative Analysis for 2020 AAPL Stock Data
# description: Notebook to load stock data and compute technical indicators using pynance and TA-Lib
# ---

# ### 1.Adjust Python Path
# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.finance_utils import load_stock_data, compute_technical_indicators

# Set plot style
sns.set_style("whitegrid")

# ### 2. Load AAPL Stock Data
aapl_file = '../data/stock_prices_cleaned_2020.csv'
aapl_df = load_stock_data(aapl_file)
if aapl_df is None:
    raise SystemExit("Failed to load AAPL data. Check file path.")

print("=== AAPL Stock Data Sample ===")
display(aapl_df.head())

# ### 3. Load Additional Financial Data (S&P 500)
sp500_file = '../data/sp500_historical_data_2020.csv'
sp500_df = load_stock_data(sp500_file)
if sp500_df is None:
    print("S&P 500 data not found locally. Attempting to fetch via pynance...")
    try:
        sp500_df = pn.data.get('^GSPC', start='2020-01-01', end='2020-12-31')
        sp500_df.reset_index(inplace=True)
        sp500_df['Date'] = pd.to_datetime(sp500_df['Date'], utc=True)
        sp500_df.to_csv(sp500_file, index=False)
    except Exception as e:
        print(f"Error fetching S&P 500 data: {e}")
        sp500_df = None

if sp500_df is not None:
    print("\n=== S&P 500 Data Sample ===")
    display(sp500_df.head())
else:
    print("Proceeding with AAPL data only.")

# ### 4. Compute Technical Indicators for AAPL
aapl_df = compute_technical_indicators(aapl_df, price_column='Close')
print("\n=== AAPL Data with Technical Indicators ===")
display(aapl_df[['Date', 'Close', 'SMA_20', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Middle', 'BB_Lower']].tail())

# ### 5. Visualize Technical Indicators
# Plot Close Price and SMA
plt.figure(figsize=(12, 6))
plt.plot(aapl_df['Date'], aapl_df['Close'], label='Close Price', color='blue')
plt.plot(aapl_df['Date'], aapl_df['SMA_20'], label='20-Day SMA', color='orange')
plt.title('AAPL Close Price and 20-Day SMA (2020)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_sma_2020.png')
plt.show()

# Plot RSI
plt.figure(figsize=(12, 4))
plt.plot(aapl_df['Date'], aapl_df['RSI_14'], label='RSI (14)', color='purple')
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')
plt.title('AAPL Relative Strength Index (2020)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_rsi_2020.png')
plt.show()

# Plot MACD
plt.figure(figsize=(12, 4))
plt.plot(aapl_df['Date'], aapl_df['MACD'], label='MACD', color='blue')
plt.plot(aapl_df['Date'], aapl_df['MACD_Signal'], label='Signal Line', color='orange')
plt.title('AAPL MACD (2020)')
plt.xlabel('Date')
plt.ylabel('MACD')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/aapl_macd_2020.png')
plt.show()

# ### 6. Compare AAPL with S&P 500 (if available)
if sp500_df is not None:
    # Normalize prices for comparison
    aapl_normalized = aapl_df['Close'] / aapl_df['Close'].iloc[0]
    sp500_normalized = sp500_df['Close'] / sp500_df['Close'].iloc[0]
    
    plt.figure(figsize=(12, 6))
    plt.plot(aapl_df['Date'], aapl_normalized, label='AAPL Normalized', color='blue')
    plt.plot(sp500_df['Date'], sp500_normalized, label='S&P 500 Normalized', color='red')
    plt.title('AAPL vs. S&P 500 Normalized Prices (2020)')
    plt.xlabel('Date')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('plots/aapl_vs_sp500_2020.png')
    plt.show()

# ### 7. Save Results
aapl_df.to_csv('../data/aapl_with_indicators_2020.csv', index=False)